### Microsoft Agent Framework - Part-3

#### Running the agent with a multi-turn conversation

- Agents are stateless and do not maintain any state internally between calls.

- To have a multi-turn conversation with an agent, you need to create an object to hold the conversation state and pass this object to the agent when running it.

- To create the conversation state object, call the get_new_thread() method on the agent instance.

![alt text](<Images/thread management.png>)


In [18]:

import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions="You are good at telling jokes.",
    name="Joker"
)

In [19]:

thread = agent.get_new_thread()

In [20]:

result1 = await agent.run("Tell me a joke about a pirate.", thread=thread)
print(result1.text)

Why did the pirate go to school?

Because he wanted to improve his "arrrrrr-ticulation!"


In [21]:

result2 = await agent.run("Now add some emojis to the joke and tell it in the voice of a pirate's parrot.", thread=thread)
print(result2.text)

🦜 "Why did the pirate go to school? 🏴‍☠️ 

Because he wanted to improve his 'arrrrrr-ticulation!' 🎓🐦"


In [26]:

import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from PyPDF2 import PdfReader

# Read the PDF file and extract text
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Path to the PDF file in the data folder
pdf_path = "data/invoice.pdf"
data_source_text = extract_text_from_pdf(pdf_path)

[2025-11-23 17:39:33 - d:\Youtube\Practice-Projects\Microsoft-Agent-Framework\.venv\Lib\site-packages\PyPDF2\_utils.py:402 - WARNING] unknown widths : 
[0, IndirectObject(4914, 0, 2084386803536), 2, 3, 249, 4, 26, 630, 27, IndirectObject(4915, 0, 2084386803536), 29, 34, 688, 35, 38, 723, 39, 56, 609, 57, 58, 594, 59, IndirectObject(4916, 0, 2084386803536), 60, 67, 703, 68, 71, 757, 72, IndirectObject(4917, 0, 2084386803536), 74, 84, 339, 85, 86, 326, 87, 89, 655, 90, 95, 588, 96, IndirectObject(4918, 0, 2084386803536), 97, 100, 706, 101, IndirectObject(4919, 0, 2084386803536), 103, 126, 741, 127, IndirectObject(4920, 0, 2084386803536), 131, 134, 648, 135, 140, 540, 141, IndirectObject(4921, 0, 2084386803536), 143, 147, 621, 148, 167, 682, 168, IndirectObject(4922, 0, 2084386803536), 169, 173, 905, 174, IndirectObject(4923, 0, 2084386803536), 175, 182, 591, 183, 186, 594, 187, IndirectObject(4924, 0, 2084386803536), 193, 215, 498, 216, IndirectObject(4925, 0, 2084386803536), 218, 223, 4

In [27]:
data_source_text

' Tax Invoice/Bill of Supply/Cash Memo\n(Original for Recipient)\n*ASSPL-Amazon Seller Services Pvt. Ltd., ARIPL-Amazon Retail India Pvt. Ltd. (only where Amazon Retail India Pvt. Ltd. fulfillment center is co-located) \nCustomers desirous of availing input GST credit are requested to create a Business account and purchase on Amazon.in/business from Business eligible offers \nPlease note that this invoice is not a demand for payment\nPage 1 of 1For Appario Retail Private Ltd:\nAuthorized Signatory\nOrder Number:  402-5005041-4753952 Invoice Number :  CCU1-4632921\nOrder Date:  04.02.2022 Invoice Details :  WB-CCU1-1034-2122\nInvoice Date :  04.02.2022\nSl.\nNoDescription Unit Price QtyNet\nAmountTax\nRateTax\nTypeTax\nAmountTotal\nAmount\n1\n2Apple iPhone 13 (128GB) - (Product) RED |\nB09G99CW2N ( B09G99CW2N ) \nHSN:85171300\nSamsung Galaxy Buds Pro (Phantom \nBlack) | XYZ123TEST₹63,474.58\n₹90001\n1₹63,474.58\n₹9000GST₹11425.42₹74,900.00\n₹10620 18%GST₹1620\nTOTAL: ₹13045.42₹85520.00\

In [28]:

# Create the agent
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions=f""" You are a helpful customer support agent. 
                    Use the following information to answer user questions:
                    {data_source_text}""",
    name="SupportBot"
)

In [32]:

prod1_thread = agent.get_new_thread()
prod2_thread = agent.get_new_thread()

In [33]:

p1_q1 = await agent.run("What is the first purchased product in this invoice?", thread=prod1_thread)
print("Prod 1:", p1_q1.text)

Prod 1: The first purchased product in this invoice is the Apple iPhone 13 (128GB) in (Product) RED, with the HSN code 85171300.


In [34]:

p1_q2 = await agent.run("What is the price of that product?", thread=prod1_thread)
print("Prod 1:", p1_q2.text)

Prod 1: The price of the Apple iPhone 13 (128GB) - (Product) RED is ₹63,474.58.


In [35]:

p2_q1 = await agent.run("What is the second purchased product in this invoice?", thread=prod2_thread)
print("Prod 2:", p2_q1.text)

Prod 2: The second purchased product in this invoice is the Samsung Galaxy Buds Pro (Phantom Black) with the code XYZ123TEST.


In [36]:

p2_q2 = await agent.run("What is the price of that product?", thread=prod2_thread)
print("Prod 2:", p2_q2.text)

Prod 2: The price of the Samsung Galaxy Buds Pro (Phantom Black) is ₹10,620.


**Testing**

In [37]:


p1_test = await agent.run("What was the product we talked about", thread=prod1_thread)
print("Prod 1:", p1_test.text)

Prod 1: The product we talked about is the Apple iPhone 13 (128GB) in (Product) RED.


In [38]:


p2_test = await agent.run("What was the product we talked about", thread=prod2_thread)
print("Prod 2:", p2_test.text)

Prod 2: The product we talked about is the Samsung Galaxy Buds Pro (Phantom Black).
